In [18]:
# Gather data for analysis - dependencies
import pandas as pd

# Define path of data files
school_csv = "Resources/schools_complete.csv"
student_csv = "Resources/students_complete.csv"

# Read files into new data frames
school_df = pd.read_csv(school_csv)
student_df = pd.read_csv(student_csv)

# Merge data from school and student data frames into new data frame - removing duplicate columns
school_student_df = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])

school_student_df.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [19]:
### Create summary of school district data

# Calculate total number of schools
total_schools = len(school_student_df["school_name"].unique())

# Calculate total number of students
total_students = school_student_df["Student ID"].count()

# Calculate total budget
total_budget_unique = school_student_df["budget"].unique()
total_budget = total_budget_unique.sum()

# Calculate average math and reading scores
avg_math_score = school_student_df["math_score"].mean()
avg_reading_score = school_student_df["reading_score"].mean()

# Calculate number of students with passing scores
students_passing_math = len(school_student_df[school_student_df["math_score"]>=70])
percent_passing_math = students_passing_math/float(total_students)*100

students_passing_reading = len(school_student_df[school_student_df["reading_score"]>=70])
percent_passing_reading = students_passing_reading/float(total_students)*100

students_passing_all = len(school_student_df[(school_student_df["math_score"]>=70) &
                                         (school_student_df["reading_score"]>=70)])
percent_passing_all = students_passing_all/float(total_students)*100

# Create data frame for summarizing district data
district_summary_df = pd.DataFrame({"Total Schools": [total_schools], 
                                    "Total Students": [total_students], 
                                    "Total Budget": [total_budget],
                                    "Average Math Score": [avg_math_score], 
                                    "Average Reading Score": [avg_reading_score],
                                    "% Passing Math": [percent_passing_math],
                                    "% Passing Reading": [percent_passing_reading],
                                    "% Passing Overall": [percent_passing_all]})

# Format data
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format) 
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.2f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.2f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.2f}%".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.2f}%".format)
district_summary_df["% Passing Overall"] = district_summary_df["% Passing Overall"].map("{:.2f}%".format)

district_summary_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,"39,170","$24,649,428",78.99,81.88,74.98%,85.81%,65.17%


In [20]:
### Create School Summary

# Add school name and type columns to new dataframe and set index to school name
school_index = school_df.set_index(["school_name"])["type"]

# Count of school rows to use for total students
school_count = school_student_df["school_name"].value_counts()

# Calculate the total budget info
school_budget = school_student_df.groupby(["school_name"]).max()["budget"]

# Calculate per student budget
student_budget = (school_budget/school_count).astype(int)

# Calculate average math and reading scores
school_math_avg = school_student_df.groupby(["school_name"]).mean()["math_score"]
school_read_avg = school_student_df.groupby(["school_name"]).mean()["reading_score"]

# Calculate passing math, reading and overall percentages

math_pass = school_student_df[school_student_df['math_score'] >= 70].groupby('school_name')['math_score'].count()
math_pass_percent = math_pass/school_count*100

reading_pass = school_student_df[school_student_df['reading_score'] >= 70].groupby('school_name')['reading_score'].count()
reading_pass_percent = reading_pass/school_count*100

all_pass =school_student_df[(school_student_df["math_score"] >= 70) &
            (school_student_df["reading_score"] >= 70)].groupby(["school_name"]).count()["student_name"]  

all_pass_percent = all_pass/school_count*100

# Create dataframe to summarize school information
school_summary_df = pd.DataFrame({"School Type":school_index,
                                "Total Students": school_count,
                                "Total Budget": school_budget,
                                "Per Student Budget":student_budget,  
                                "Average Math Score": school_math_avg, 
                                "Average Reading Score": school_read_avg,
                                "% Passing Math": math_pass_percent,
                                "% Passing Reading": reading_pass_percent,
                                "% Passing Overall": all_pass_percent})


# Format data
school_summary_df["Total Students"] = school_summary_df["Total Students"].map("{:,}".format) 
school_summary_df["Total Budget"] = school_summary_df["Total Budget"].map("${:,}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("${:}".format)
school_summary_df["Average Math Score"] = school_summary_df["Average Math Score"].map("{:.2f}".format)
school_summary_df["Average Reading Score"] =school_summary_df["Average Reading Score"].map("{:.2f}".format)
school_summary_df["% Passing Math"] = school_summary_df["% Passing Math"].map("{:.2f}%".format)
school_summary_df["% Passing Reading"] = school_summary_df["% Passing Reading"].map("{:.2f}%".format)
school_summary_df["% Passing Overall"] = school_summary_df["% Passing Overall"].map("{:.2f}%".format)

school_summary_df

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Bailey High School,District,"4,976","$3,124,928",$628,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916",$644,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,90.54%


In [21]:
### Top Performing Schools by % Overall Passing

# Create new data frame for viewing top 5 peforming schools by % Overall Passing using school_summary_df
# created in cell 3 for the School Summary

top_passing_performers = school_summary_df.sort_values("% Passing Overall",ascending=False)
top_passing_performers.head()

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130",$638,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574",$578,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,90.54%


In [22]:
### Bottom Performing Schools by % Overall Passing

# Create new data frame for viewing bottom 5 peforming schools by % Overall Passing using school_summary_df
# created in cell 3 for the School Summary

bottom_passing_performers = school_summary_df.sort_values("% Passing Overall")
bottom_passing_performers.head()

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Rodriguez High School,District,"3,999","$2,547,363",$637,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%


In [23]:
### Math Scores by Grade

# Pull necessary columns for calculations from original school_student_df.
math_score_columns = ["school_name","grade","math_score"]
math_score_df = school_student_df[math_score_columns]

# Set index to school name
math_index = math_score_df.set_index(["school_name"])

# Locate and pull data for each of the 4 grades into a series
nineth_grade = math_score_df.loc[math_score_df["grade"] == "9th",:]
tenth_grade = math_score_df.loc[math_score_df["grade"] == "10th",:]
eleventh_grade = math_score_df.loc[math_score_df["grade"] == "11th",:]
twelfth_grade = math_score_df.loc[math_score_df["grade"] == "12th",:]

# Group grade data by school
nineth_math_avg = nineth_grade.groupby(["school_name"]).mean()["math_score"]
tenth_math_avg = tenth_grade.groupby(["school_name"]).mean()["math_score"]
eleventh_math_avg = eleventh_grade.groupby(["school_name"]).mean()["math_score"]
twelfth_math_avg = twelfth_grade.groupby(["school_name"]).mean()["math_score"]

# Create data frame for results
avg_math_score_df = pd.DataFrame({"9th":nineth_math_avg,
                                "10th": tenth_math_avg,
                                "11th": eleventh_math_avg,
                                "12th":twelfth_math_avg})

# Format data
avg_math_score_df.index.name=None
avg_math_score_df["9th"] = avg_math_score_df["9th"].map("{:.2f}%".format) 
avg_math_score_df["10th"] = avg_math_score_df["10th"].map("{:.2f}%".format) 
avg_math_score_df["11th"] = avg_math_score_df["11th"].map("{:.2f}%".format) 
avg_math_score_df["12th"] = avg_math_score_df["12th"].map("{:.2f}%".format) 


avg_math_score_df                               

,9th,10th,11th,12th
Bailey High School,77.08%,77.00%,77.52%,76.49%
Cabrera High School,83.09%,83.15%,82.77%,83.28%
Figueroa High School,76.40%,76.54%,76.88%,77.15%
Ford High School,77.36%,77.67%,76.92%,76.18%
Griffin High School,82.04%,84.23%,83.84%,83.36%
Hernandez High School,77.44%,77.34%,77.14%,77.19%
Holden High School,83.79%,83.43%,85.00%,82.86%
Huang High School,77.03%,75.91%,76.45%,77.23%
Johnson High School,77.19%,76.69%,77.49%,76.86%
Pena High School,83.63%,83.37%,84.33%,84.12%


In [24]:
### Reading Scores by Grade

# Pull necessary columns for calculations from original school_student_df.
reading_score_columns = ["school_name","grade","reading_score"]
reading_score_df = school_student_df[reading_score_columns]

# Set index to school name
reading_index = reading_score_df.set_index(["school_name"])

# Locate and pull data for each of the 4 grades into a series
nineth_grade_r = reading_score_df.loc[reading_score_df["grade"] == "9th",:]
tenth_grade_r = reading_score_df.loc[reading_score_df["grade"] == "10th",:]
eleventh_grade_r = reading_score_df.loc[reading_score_df["grade"] == "11th",:]
twelfth_grade_r = reading_score_df.loc[reading_score_df["grade"] == "12th",:]

# Group grade data by school
nineth_reading_avg = nineth_grade_r.groupby(["school_name"]).mean()["reading_score"]
tenth_reading_avg = tenth_grade_r.groupby(["school_name"]).mean()["reading_score"]
eleventh_reading_avg = eleventh_grade_r.groupby(["school_name"]).mean()["reading_score"]
twelfth_reading_avg = twelfth_grade_r.groupby(["school_name"]).mean()["reading_score"]

# Create data frame for results
avg_reading_score_df = pd.DataFrame({"9th":nineth_reading_avg,
                                    "10th": tenth_reading_avg,
                                    "11th": eleventh_reading_avg,
                                    "12th":twelfth_reading_avg})

# Format data
avg_reading_score_df.index.name=None
avg_reading_score_df["9th"] = avg_reading_score_df["9th"].map("{:.2f}%".format) 
avg_reading_score_df["10th"] = avg_reading_score_df["10th"].map("{:.2f}%".format) 
avg_reading_score_df["11th"] = avg_reading_score_df["11th"].map("{:.2f}%".format) 
avg_reading_score_df["12th"] = avg_reading_score_df["12th"].map("{:.2f}%".format) 


avg_reading_score_df         

,9th,10th,11th,12th
Bailey High School,81.30%,80.91%,80.95%,80.91%
Cabrera High School,83.68%,84.25%,83.79%,84.29%
Figueroa High School,81.20%,81.41%,80.64%,81.38%
Ford High School,80.63%,81.26%,80.40%,80.66%
Griffin High School,83.37%,83.71%,84.29%,84.01%
Hernandez High School,80.87%,80.66%,81.40%,80.86%
Holden High School,83.68%,83.32%,83.82%,84.70%
Huang High School,81.29%,81.51%,81.42%,80.31%
Johnson High School,81.26%,80.77%,80.62%,81.23%
Pena High School,83.81%,83.61%,84.34%,84.59%


In [25]:
### Scores by School Spending

# Create new data frame from School Summary data frame with only needed columns

school_spend_df = pd.DataFrame({"Per Student Budget":student_budget,
                                "Average Math Score": school_math_avg, 
                                "Average Reading Score": school_read_avg,
                                "% Passing Math": math_pass_percent,
                                "% Passing Reading": reading_pass_percent,
                                "% Passing Overall": all_pass_percent})

# Create bins to hold spending ranges
school_spend_bins = [0, 584, 629, 644, 680]

# Create bin labels
school_spend_labels = ["< $585","$585-630","$630-645","$645-680"]

# Place data into bins
school_spend_df["Spending Ranges (per Student)"] = pd.cut(school_spend_df["Per Student Budget"]
                                ,school_spend_bins,labels = school_spend_labels,include_lowest = True)
# Set index on dataframe
school_spend_df = school_spend_df.set_index(["Spending Ranges (per Student)"])

# Group school_spend_df by the binned Spending Ranges (per Student)
spend_per_student = school_spend_df.groupby(["Spending Ranges (per Student)"]).mean()[
    ["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Passing Overall"]]

# Formatting
spend_per_student["Average Math Score"] = spend_per_student["Average Math Score"].map("{:.2f}".format)
spend_per_student["Average Reading Score"] = spend_per_student["Average Reading Score"].map("{:.2f}".format)
spend_per_student["% Passing Math"] = spend_per_student["% Passing Math"].map("{:.2f}%".format)
spend_per_student["% Passing Reading"] = spend_per_student["% Passing Reading"].map("{:.2f}%".format)
spend_per_student["% Passing Overall"] = spend_per_student["% Passing Overall"].map("{:.2f}%".format)
    
spend_per_student


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Spending Ranges (per Student),,,,,
< $585,83.46,83.93,93.46%,96.61%,90.37%
$585-630,81.90,83.16,87.13%,92.72%,81.42%
$630-645,78.52,81.62,73.48%,84.39%,62.86%
$645-680,77.00,81.03,66.16%,81.13%,53.53%


In [26]:
### Scores by School Size

# Create new data frame from School Summary data frame with only needed columns

school_size_df = pd.DataFrame({"Total Students": school_count,
                                "Average Math Score": school_math_avg, 
                                "Average Reading Score": school_read_avg,
                                "% Passing Math": math_pass_percent,
                                "% Passing Reading": reading_pass_percent,
                                "% Passing Overall": all_pass_percent})

# Create bins to hold spending ranges
school_size_bins = [0, 999, 1999, 5000]

# Create bin labels
school_size_labels = ["Small < 1,000","Medium (1,000-2,000)","Large (2,000-5,000)"]

# Place data into bins
school_size_df["School Size"] = pd.cut(school_size_df["Total Students"],school_size_bins,
                                                    labels = school_size_labels,include_lowest = True)
# Set index on dataframe
school_size_df = school_size_df.set_index(["School Size"])

# Group school_size_df by the binned school size ranges
school_size = school_size_df.groupby(["School Size"]).mean()[["Average Math Score","Average Reading Score",
                                            "% Passing Math","% Passing Reading","% Passing Overall"]]

# Formatting
school_size["Average Math Score"] = school_size["Average Math Score"].map("{:.2f}".format)
school_size["Average Reading Score"] = school_size["Average Reading Score"].map("{:.2f}".format)
school_size["% Passing Math"] = school_size["% Passing Math"].map("{:.2f}%".format)
school_size["% Passing Reading"] = school_size["% Passing Reading"].map("{:.2f}%".format)
school_size["% Passing Overall"] = school_size["% Passing Overall"].map("{:.2f}%".format)
school_size



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
"Small < 1,000",83.82,83.93,93.55%,96.10%,89.88%
"Medium (1,000-2,000)",83.37,83.86,93.60%,96.79%,90.62%
"Large (2,000-5,000)",77.75,81.34,69.96%,82.77%,58.29%


In [27]:
### Scores by School Type

school_type_df = pd.DataFrame({"School Type":school_index,
                                "Average Math Score": school_math_avg, 
                                "Average Reading Score": school_read_avg,
                                "% Passing Math": math_pass_percent,
                                "% Passing Reading": reading_pass_percent,
                                "% Passing Overall": all_pass_percent})

# Set index on dataframe
school_type_df = school_type_df.set_index(["School Type"])

# Group school type 
school_type = school_type_df.groupby(["School Type"]).mean()[["Average Math Score","Average Reading Score",
                                            "% Passing Math","% Passing Reading","% Passing Overall"]]

# Formatting
school_type["Average Math Score"] = school_type["Average Math Score"].map("{:.2f}".format)
school_type["Average Reading Score"] = school_type["Average Reading Score"].map("{:.2f}".format)
school_type["% Passing Math"] = school_type["% Passing Math"].map("{:.2f}%".format)
school_type["% Passing Reading"] = school_type["% Passing Reading"].map("{:.2f}%".format)
school_type["% Passing Overall"] = school_type["% Passing Overall"].map("{:.2f}%".format)


school_type


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%
